# Downloading data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications.efficientnet import preprocess_input

# ==== Paths ====
train_dir = '/content/drive/MyDrive/VGG/train'
val_dir   = '/content/drive/MyDrive/VGG/valid'
test_dir  = '/content/drive/MyDrive/VGG/test'

# ==== Parameters ====
img_size    = (256, 256)
batch_size  = 16
seed        = 123
num_classes = 1   # binary classification


In [3]:
def prepare_ds(path):
    ds = tf.keras.utils.image_dataset_from_directory(
        path, seed=seed,
        image_size=img_size, batch_size=batch_size,
        label_mode='int'
    )
    ds = ds.map(lambda x, y: (preprocess_input(tf.cast(x, tf.float32)), y))
    return ds.prefetch(tf.data.AUTOTUNE)

train_ds = prepare_ds(train_dir)
val_ds   = prepare_ds(val_dir)
test_ds  = prepare_ds(test_dir)


Found 3200 files belonging to 2 classes.
Found 398 files belonging to 2 classes.
Found 402 files belonging to 2 classes.


In [4]:
def build_model(backbone_class):
    backbone = backbone_class(
        input_shape=img_size + (3,),
        include_top=False,
        weights='imagenet'
    )
    backbone.trainable = False

    inputs = layers.Input(shape=img_size + (3,))
    x = backbone(inputs, training=False)
    x = layers.GlobalAveragePooling2D(name='gap')(x)
    x = layers.Dropout(0.3, name='dropout1')(x)
    x = layers.Dense(
        128, activation='relu',
        kernel_regularizer=regularizers.l2(1e-4),
        name='dense128'
    )(x)
    x = layers.Dropout(0.3, name='dropout2')(x)
    outputs = layers.Dense(1, activation='sigmoid', name='output')(x)

    model = models.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.AdamW(learning_rate=2.5e-4, weight_decay=1e-4),
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.AUC(name='auc'), 'accuracy']
    )
    return model


In [5]:
from tensorflow.keras.applications import InceptionV3
model = build_model(InceptionV3)
model.summary()


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_v3 (Functional)       │ (None, 6, 6, 2048)     │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gap (GlobalAveragePooling2D)    │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout1 (Dropout)              │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense128 (Dense)                │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout2 (Dropout)              │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,065,185 (84.17 MB)

 Trainable params: 262,401 (1.00 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [7]:
from tensorflow.keras.callbacks import (
    ModelCheckpoint, EarlyStopping,
    ReduceLROnPlateau, CSVLogger
)

callbacks = [
    ModelCheckpoint('best_model.h5', monitor='val_auc', save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_auc', patience=10, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1),
    CSVLogger('training_log.csv', append=False)
]

history = model.fit(
    train_ds, validation_data=val_ds,
    epochs=50, callbacks=callbacks
)


Epoch 1/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.5184 - auc: 0.5312 - loss: 6.4191
Epoch 1: val_auc improved from inf to 0.57648, saving model to best_model.h5


200/200 ━━━━━━━━━━━━━━━━━━━━ 882s 4s/step - accuracy: 0.5185 - auc: 0.5312 - loss: 6.4066 - val_accuracy: 0.5050 - val_auc: 0.5765 - val_loss: 0.7178 - learning_rate: 2.5000e-04
Epoch 2/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.5492 - auc: 0.5632 - loss: 0.9600
Epoch 2: val_auc improved from 0.57648 to 0.56557, saving model to best_model.h5


200/200 ━━━━━━━━━━━━━━━━━━━━ 70s 151ms/step - accuracy: 0.5492 - auc: 0.5632 - loss: 0.9595 - val_accuracy: 0.5578 - val_auc: 0.5656 - val_loss: 0.7047 - learning_rate: 2.5000e-04
Epoch 3/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - accuracy: 0.5412 - auc: 0.5484 - loss: 0.7685
Epoch 3: val_auc did not improve from 0.56557
200/200 ━━━━━━━━━━━━━━━━━━━━ 39s 141ms/step - accuracy: 0.5413 - auc: 0.5485 - loss: 0.7684 - val_accuracy: 0.5829 - val_auc: 0.6859 - val_loss: 0.6920 - learning_rate: 2.5000e-04
Epoch 4/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - accuracy: 0.5767 - auc: 0.6144 - loss: 0.7049
Epoch 4: val_auc did not improve from 0.56557
200/200 ━━━━━━━━━━━━━━━━━━━━ 43s 153ms/step - accuracy: 0.5767 - auc: 0.6144 - loss: 0.7049 - val_accuracy: 0.6231 - val_auc: 0.7031 - val_loss: 0.6718 - learning_rate: 2.5000e-04
Epoch 5/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.5540 - auc: 0.5970 - loss: 0.6973
Epoch 5: val_auc did not improve from 0.56557
200/200 ━━━━━━━━

In [8]:
import numpy as np
from sklearn.metrics import roc_curve, accuracy_score, roc_auc_score, f1_score
import tensorflow as tf
# Evaluate on test set using the best saved model
model.load_weights('best_model.h5')

# 1) Predict probabilities
y_true, y_prob = [], []
for x_batch, y_batch in test_ds:
    y_true.extend(y_batch.numpy())
    y_prob.extend(model.predict(x_batch).flatten())
y_true = np.array(y_true)
y_prob = np.array(y_prob)

# 2) Compute ROC curve and Youden’s J threshold
fpr, tpr, thresholds = roc_curve(y_true, y_prob)
j_idx = np.argmax(tpr - fpr)
best_threshold = thresholds[j_idx]
print(f'Optimal threshold (Youden J): {best_threshold:.3f}')

# 3) Compute binary predictions and metrics
y_pred = (y_prob >= best_threshold).astype(int)
test_acc = accuracy_score(y_true, y_pred)
test_auc = roc_auc_score(y_true, y_prob)
test_f1  = f1_score(y_true, y_pred)

print(f'Test AUC : {test_auc:.4f}')
print(f'Test Acc : {test_acc:.4f}')
print(f'Test F1  : {test_f1:.4f}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
Opt